!pip3 uninstall -y torch torchvision

!pip3 install torch torchvision

In [ ]:
import torch
# import the data
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

In [ ]:
import os
os.environ["KERAS_BACKEND"] = "torch"

In [ ]:
import numpy as np
import keras
from keras import layers

In [ ]:
# function to evaluate model
# returns (accuracy, tpr, tnr, f1)
# takes as input real, predicted
def genResults(real, predicted):
    real=np.argmax(real, axis=1)
    predicted=np.argmax(predicted, axis=1)

    conf_matrix = confusion_matrix(y_true=real, y_pred=predicted)
    tn, fp, fn, tp = conf_matrix.ravel()

    f1 = (2*tp)/((2*tp)+fp+fn)

    acc = (tp + tn) / (tp + tn + fn + fp)

    tpr = tp / (tp + fp)
    tnr = tn / (tn + fn)

    return (acc, tpr, tnr, f1)

In [ ]:
# split it like above

# do the shaping if necessary
# change the type of the columns that are characters to one-hot-encodings or something like that

# make a model with one layer
# run the model and see what happens

In [ ]:
df = pd.read_csv('data\\online_shoppers_intention.csv')

print(df)

In [ ]:
df['total_duration'] = df['Administrative_Duration'] + df['Informational_Duration'] + df['ProductRelated_Duration']
df['proportion_of_duration'] = df['ProductRelated_Duration'] / df['total_duration']

In [ ]:
df.drop(columns=['Informational_Duration', 'Administrative_Duration', 
                 'ProductRelated_Duration', 'Administrative', 'Informational', 
                 'ProductRelated', 'OperatingSystems', 'Browser'])

In [ ]:
# df['Month'].replace(['
print(df['Month'].unique())

In [ ]:
df['Month'].replace(['Jan','Feb','Mar','Apr','May','June','Jul','Aug','Sep','Oct','Nov','Dec'],
                    [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], inplace=True)

In [ ]:
print(df['Month'].unique())

In [ ]:
print(df['VisitorType'].unique())

In [ ]:
df['VisitorType'].replace(['Returning_Visitor', 'New_Visitor', 'Other'], [0, 1, 2], inplace=True)

In [ ]:
df_copy = df.copy(deep=True)

In [ ]:
df_copy['Weekend'].replace([False, True], [0,1], inplace=True)

In [ ]:
df_copy['Revenue'].replace([False, True], [0,1], inplace=True)

In [ ]:
df_copy = df_copy.drop(columns=['Informational_Duration', 'Administrative_Duration', 
                 'ProductRelated_Duration', 'Administrative', 'Informational', 
                 'ProductRelated', 'OperatingSystems', 'Browser'])

In [ ]:
df_copy.fillna(1, inplace=True)

In [ ]:
df_numpy = df_copy.to_numpy()

In [ ]:

df_numpy.shape[0]
testSample = np.random.choice(df_numpy.shape[0], math.floor(0.7 * df_numpy.shape[0]), replace=False)

In [ ]:
print(testSample)

In [ ]:
print(df_numpy[testSample])

train = df_numpy[testSample]
test = df_numpy[~testSample]

print("Train length: " + str(train.shape[0]))
print("Test lenght: " + str(test.shape[0]))

In [ ]:
df_copy_y = df_copy['Revenue']
df_copy_x = df_copy.loc[:, df_copy.columns != 'Revenue']

#print(df_copy_y)
#print(df_copy_x)

In [ ]:
df_copy_x_train, df_copy_x_test, df_copy_y_train, df_copy_y_test = train_test_split(
    df_copy_x, df_copy_y, test_size=0.3, random_state=42)

In [ ]:
numpy_x_train = df_copy_x_train.to_numpy()
numpy_x_test = df_copy_x_test.to_numpy()
numpy_y_train = df_copy_y_train.to_numpy()
numpy_y_test = df_copy_y_test.to_numpy()

num_classes = 2

input_shape = (11, 1)

# Make sure images have shape (28, 28, 1)
numpy_x_train = np.expand_dims(numpy_x_train, -1)
numpy_x_test = np.expand_dims(numpy_x_test, -1)
print("x_train shape:", numpy_x_train.shape)
print(numpy_x_train.shape[0], "train samples")
print(numpy_x_test.shape[0], "test samples")

# convert class vectors to binary class matrices
numpy_y_train = keras.utils.to_categorical(numpy_y_train, num_classes)
numpy_y_test = keras.utils.to_categorical(numpy_y_test, num_classes)

print("y_train shape: ", numpy_y_train.shape)

In [ ]:
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Dense(20, activation="relu"),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model.summary()

In [ ]:
batch_size = 128
epochs = 30

model.compile(loss="categorical_crossentropy", optimizer="adam", 
              metrics=["accuracy"])

model.fit(numpy_x_train, numpy_y_train, batch_size=batch_size, 
          epochs=epochs, validation_split=0.1)

In [ ]:
score = model.evaluate(numpy_x_test, numpy_y_test, verbose=1)

In [ ]:
predicted = model.predict(numpy_x_test)

In [ ]:
print("numpy_y_test shape: ", numpy_y_test.shape)
print(numpy_y_test)
print("predicted: ", predicted.shape)
print(predicted)

In [ ]:
acc, tpr, tnr, f1 = genResults(numpy_y_test, predicted)

print("Accuracy: ", acc)
print("True-positive rate: ", tpr)
print("True-negative rate: ", tnr)
print("F1 Score: ", f1)